In [1]:
import tensorflow as tf
import torch
import torchsummary



2023-07-25 13:32:55.054894: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 13:32:55.056372: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 13:32:55.086405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 13:32:55.087239: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 13:32:55.687165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from networks.att_squeeze_unet import AttSqueezeUNet as AttSqueezeUNetTF
model_tf = AttSqueezeUNetTF(2,True)
random_input_tf = tf.random.normal((1, 384, 512, 3))
model_tf(random_input_tf)
model_tf.summary()

Tf x5 shape (1, 24, 32, 512)
Tf x4 shape (1, 24, 32, 384)
Tf d5 shape (1, 24, 32, 384)
Tf d4 shape (1, 24, 32, 256)
Tf d3 shape (1, 24, 32, 256)
Tf x2 shape (1, 48, 64, 128)
Tf d3 shape (1, 48, 64, 128)
Tf d2 shape (1, 96, 128, 64)
Tf d1 shape (1, 192, 256, 64)
Model: "AttSqueezeUNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1792      
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
  (FireModule)               multiple                  11472     
                                                                 
  (FireModule)               multiple                  12496     
                                                      

Tf d3 shape (1, 24, 32, 256)
Tf x2 shape (1, 48, 64, 128)

In [4]:
from networks.att_squeeze_unet import UpsamplingBlock as UpsamplingBlockTF
from networks.att_squeeze_unet_torch import UpsamplingBlock as UpsamplingBlockTorch
from torch import nn
tf_upsampling_block = UpsamplingBlockTF(
            filters=64,
            fire_id=11,
            squeeze=16,
            expand=64,
            strides=(2, 2),
            deconv_ksize=3,
            att_filters=16,
        )
random_input_x = tf.random.uniform((1, 24, 32, 256))
random_input_g = tf.random.uniform((1, 48, 64, 128))
print(tf_upsampling_block(random_input_x, random_input_g).shape)
random_input_x_torch = torch.rand((1, 512, 24, 32))
random_input_g_torch = torch.rand((1, 384, 48, 64))
torch_upsampling_block = UpsamplingBlockTorch(
    filters=64,
    fire_id=9,
    squeeze=16,
    expand=64,
    deconv_ksize=3,
    strides=(2, 2),
    att_filters=16,
    x_input_shape=random_input_x_torch.shape,
    g_input_shape=random_input_g_torch.shape,
    )

torch_upsampling_block(random_input_x_torch, random_input_g_torch).shape



(1, 48, 64, 128)
Upconv in shape torch.Size([1, 512, 24, 32])
Upconv shape torch.Size([1, 64, 48, 64])
Attention shape torch.Size([1, 384, 48, 64])
Upconv in shape torch.Size([1, 512, 24, 32])
Upconv shape torch.Size([1, 64, 48, 64])
Attention shape torch.Size([1, 384, 48, 64])


torch.Size([1, 128, 48, 64])

In [10]:
upconv = nn.ConvTranspose2d(
            in_channels=random_input_x_torch.shape[1],
            out_channels=64,
            kernel_size=3,
            stride=(2,2),
            padding=(1, 1),
            # output_padding=(1, 1),
        )
upconv(random_input_x_torch).shape

torch.Size([1, 64, 71, 95])

In [5]:
out = tf_upsampling_block.upconv(random_input_x)
from matplotlib import pyplot as plt
plt.show()

TensorShape([1, 48, 64, 64])

In [ ]:
from networks.att_squeeze_unet import AttentionBlock as AttentionBlockTF 
from networks.att_squeeze_unet_torch import AttentionBlock as AttentionBlockTorch
tf_attention_block = AttentionBlockTF(32)
random_input = tf.random.uniform((1, 256, 256, 32))
random_input_1 = tf.random.uniform((1, 256, 256, 16))
tf_attention_block(random_input_1, random_input)
random_input_torch = torch.rand((1, 32, 256, 256))
random_input_1_torch = torch.rand((1, 16, 256, 256))
torch_attention_block = AttentionBlockTorch(32, 16, 32)
torch_attention_block(random_input_torch, random_input_1_torch)

In [ ]:
from networks.att_squeeze_unet import FireModule as FireModuleTF 
from networks.att_squeeze_unet_torch import FireModule as FireModuleTorch
squeeze = 16
expand = 64
tf_fire_module = FireModuleTF(1, squeeze, expand)
torch_fire_module = FireModuleTorch(1, squeeze, expand)
random_input = tf.random.uniform((1, 256, 256, 3))
random_input_torch = torch.rand((1, 3, 256, 256))
torchsummary.summary(torch_fire_module, (3, 256, 256))
tf_fire_module(random_input)
tf_fire_module.summary()